In [39]:
import re
import os
import parse
import pdfplumber
import pandas as pd
from openpyxl import load_workbook
import xlsxwriter
from collections import namedtuple
from pathlib import Path

In [40]:
def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,
                       truncate_sheet=False, 
                       **to_excel_kwargs):
    # Excel file doesn't exist - saving and exiting
    if not os.path.isfile(filename):
        df.to_excel(
            filename,
            sheet_name=sheet_name, 
            startrow=startrow if startrow is not None else 0, 
            **to_excel_kwargs)
        return
    
    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    writer = pd.ExcelWriter(filename, engine='openpyxl', mode='a')

    # try to open an existing workbook
    writer.book = load_workbook(filename)
    
    # get the last row in the existing Excel sheet
    # if it was not specified explicitly
    if startrow is None and sheet_name in writer.book.sheetnames:
        startrow = writer.book[sheet_name].max_row

    # truncate sheet
    if truncate_sheet and sheet_name in writer.book.sheetnames:
        # index of [sheet_name] sheet
        idx = writer.book.sheetnames.index(sheet_name)
        # remove [sheet_name]
        writer.book.remove(writer.book.worksheets[idx])
        # create an empty sheet [sheet_name] using old index
        writer.book.create_sheet(sheet_name, idx)
    
    # copy existing sheets
    writer.sheets = {ws.title:ws for ws in writer.book.worksheets}

    if startrow is None:
        startrow = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, **to_excel_kwargs)

    # save the workbook
    writer.save()

In [41]:
LineCompras = namedtuple('LineCompras', 'Compras Quantidade CotaçãodeCompra CustodeCompra TaxadeLiquidacao Emolumentos CustoTotal CustoLiquido')
LineVendas = namedtuple('LineVendas', 'Vendas Quantidade CotaçãodeVenda ReceitadeVenda TaxadeLiquidacao Emolumentos IRRFnaoconta ReceitaTotal ReceitaLiquida')

In [42]:
compra_re = re.compile(r'(\d+) (\d+,\d+) (\d+,\d+) D')
venda_re = re.compile(r'(\d+) (\d+,\d+) (\d+,\d+) C')
txliq = re.compile(r'(Taxa de liquidação) (\d+,\d+)')
emol = re.compile(r'(Emolumentos) (\d+,\d+)')
irrf = re.compile(r'(IRRF s/ operações, base R)(\W)(\d+,\d+) (\d+,\d+)')

In [43]:
file = Path('Nota de Corretagem 24-06-2021.pdf')
registro = Path('historico.xlsx')

In [44]:
c_lines = []
v_lines = []

with pdfplumber.open(file, password = '511') as pdf:
    pages = pdf.pages
    for page in pdf.pages:
        text = page.extract_text()
        punc = '.'
        for ele in text: 
            if ele in punc: 
                text = text.replace(ele, "")
        for line in text.split('\n'):
            compras = compra_re.search(line)
            vendas = venda_re.search(line)

In [45]:
            if compras:
                c_ticker = re.compile(r'(1-BOVESPA C) (\w+) (\w+\W+\w+)').search(line).group(3)
                c_quant = compras.group(1)
                c_cota = compras.group(2)
                c_custo = compras.group(3)
                c_append = c_lines.append(LineCompras(c_ticker, c_quant, c_cota, c_custo, 5, 6, 7, 8))

In [46]:
            if vendas:
                v_ticker = re.compile(r'(1-BOVESPA V) (\w+) (\w+\W+\w+)').search(line).group(3)
                v_quant = vendas.group(1)
                v_cota = vendas.group(2)
                v_receita = vendas.group(3)
                v_txliq = txliq.search(text).group(2)
                v_emol = emol.search(text).group(2)
                v_irrf = irrf.search(text).group(4)
                v_lines.append(LineVendas(v_ticker, v_quant, v_cota, v_receita, v_txliq, v_emol, v_irrf, 8, 9))

In [47]:
print(compra_re)

None


In [19]:
c_df = pd.DataFrame(c_lines)
v_df = pd.DataFrame(v_lines)

In [20]:
#filename = os.path.splitext(file)[0]
#datasearch = re.compile(r'\w{2}-\w{2}-\w{4}')
#data = datasearch.search(filename)
#c_df.to_excel(data.group(0)+' Compras.xlsx', sheet_name='Compras', startrow=4, header=True, index=False)
#v_df.to_excel(data.group(0)+' Vendas.xlsx', sheet_name='Vendas', startrow=7, header=True, index=False)